Vocab

In [32]:
import pandas as pd
import numpy as  np
from gensim.models import Word2Vec
import torch
from torch.nn.utils.rnn import pad_sequence
import pickle

In [33]:
df_title = pd.read_pickle('./data/clean_title.pkl')
df_text = pd.read_pickle('./data/clean_text.pkl')
print(df_title.columns)
print(df_text.columns)
print(df_title.head())
print(df_text.head())

Index(['index', 'title', 'class'], dtype='object')
Index(['index', 'text', 'class'], dtype='object')
   index                                              title  class
0      0  [law, enforcement, high, alert, following, thr...      1
2      2  [unbelievable, obama, attorney, general, says,...      1
3      3  [bobby, jindal, raised, hindu, uses, story, ch...      0
4      4  [satan, 2, russia, unvelis, image, terrifying,...      1
5      5  [time, christian, group, sues, amazon, splc, d...      1
   index                                               text  class
0      0  [comment, expected, barack, obama, members, fy...      1
1      1                    [post, votes, hillary, already]      1
2      2  [demonstrators, gathered, last, night, exercis...      1
3      3  [dozen, politically, active, pastors, came, pr...      0
4      4  [rs, 28, sarmat, missile, dubbed, satan, 2, re...      1


In [34]:
model_text = Word2Vec(df_text["text"], vector_size=100)
model_title = Word2Vec(df_title["title"], vector_size=100)
model_text.save("./data/model_text")
model_title.save("./data/model_title")
print(model_title.corpus_total_words)

657028


KeyError: "Key 'wuz' not present"

In [35]:
model_text = Word2Vec.load("./data/model_text")
model_title = Word2Vec.load("./data/model_title")

In [36]:
# Split dataset
def split(df, test_split=0.25):
    df_test = df.sample(frac=test_split)
    df_train = df.drop(df_test.index)
    df_train.reset_index(drop=True)
    return df_test, df_train


df_text: pd.DataFrame = pd.read_pickle("./data/clean_text.pkl")

text_test, text_train = split(df_text,0.99)
print(text_test.shape)
print(text_train.shape)
print(text_train.columns)


(70625, 3)
(713, 3)
Index(['index', 'text', 'class'], dtype='object')


In [38]:
# make tensors
def text_to_tensor(text_list, word_model):
    vectors = [word_model.wv[word] for word in text_list if word in word_model.wv]
    return torch.tensor(vectors)


def stack_tensor(df: pd.DataFrame, col_name, word_model):
    tensor_ls = []
    for row in df.itertuples(index=False):
        # print(row)
        # print(row[df.columns.get_loc(col_name)])
        tensor_ls.append(text_to_tensor(row[df.columns.get_loc(col_name)], word_model))
    padded_ls = pad_sequence(tensor_ls,batch_first=True)
    return torch.stack(tuple(padded_ls))


text_train_tensor = stack_tensor(text_train, 'text', model_text)
print(text_train_tensor.size())
with open("./data/text_train_tensor","wb") as f:
    pickle.dump(text_train_tensor,f)

<built-in method size of Tensor object at 0x2b05c32e0>


torch.Size([713, 2869, 100])
